In [1]:
from keras.applications import resnet50

Using TensorFlow backend.


In [2]:

# Resnet was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 

#Loads the Resnet model 
model = resnet50.ResNet50(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

C:\Users\Education\Anaconda3\envs\baseclone\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [3]:
for layer in model.layers:
    print(layer.__class__.__name__,end='            ' )
    layer.trainable=False
    print(layer.trainable)

InputLayer            False
ZeroPadding2D            False
Conv2D            False
BatchNormalization            False
Activation            False
ZeroPadding2D            False
MaxPooling2D            False
Conv2D            False
BatchNormalization            False
Activation            False
Conv2D            False
BatchNormalization            False
Activation            False
Conv2D            False
Conv2D            False
BatchNormalization            False
BatchNormalization            False
Add            False
Activation            False
Conv2D            False
BatchNormalization            False
Activation            False
Conv2D            False
BatchNormalization            False
Activation            False
Conv2D            False
BatchNormalization            False
Add            False
Activation            False
Conv2D            False
BatchNormalization            False
Activation            False
Conv2D            False
BatchNormalization            False
Activation    

In [5]:
def add_model(bottom_model,nmclass):
    topmodel=bottom_model.output
    topmodel=Flatten(name='flatten')(topmodel)
    topmodel=Dense(399,activation='relu')(topmodel)
    topmodel=Dense(79,activation='relu')(topmodel)
    topmodel=Dense(nmclass,activation='sigmoid')(topmodel)
    return topmodel
    
    

In [6]:
from keras.models import Sequential
from keras.layers import Dense,Flatten
import cv2
from PIL import Image

model i

In [7]:
from keras.models import Model
num_class=1
Head=add_model(model,num_class)
modelnew=Model(inputs=model.input,outputs=Head)
print(modelnew.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [8]:
modelnew.output

<tf.Tensor 'dense_3/Sigmoid:0' shape=(None, 1) dtype=float32>

In [9]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = './Rahul/train/'
validation_data_dir = './Rahul/valid/'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 10
val_batchsize = 6
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='binary')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='binary',
        shuffle=False)

Found 94 images belonging to 1 classes.
Found 6 images belonging to 1 classes.


In [10]:
train_generator

In [11]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("face.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
modelnew.compile(loss = 'binary_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 100
nb_validation_samples = 16
epochs = 3
batch_size = 10

history = modelnew.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

modelnew.save("face.h5")

Epoch 1/3
10/10 [==============================] - 168s 17s/step - loss: 0.1012 - accuracy: 0.9255 - val_loss: 0.0000e+00 - val_accuracy: 1.0000

Epoch 00001: val_loss improved from inf to 0.00000, saving model to face.h5
Epoch 2/3
10/10 [==============================] - 88s 9s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000

Epoch 00002: val_loss did not improve from 0.00000
Epoch 3/3
10/10 [==============================] - 91s 9s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000

Epoch 00003: val_loss did not improve from 0.00000


In [12]:
modelnew.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

bn5a_branch2c (BatchNormalizati (None, 7, 7, 2048)   8192        res5a_branch2c[0][0]             
__________________________________________________________________________________________________
bn5a_branch1 (BatchNormalizatio (None, 7, 7, 2048)   8192        res5a_branch1[0][0]              
__________________________________________________________________________________________________
add_14 (Add)                    (None, 7, 7, 2048)   0           bn5a_branch2c[0][0]              
                                                                 bn5a_branch1[0][0]               
__________________________________________________________________________________________________
activation_43 (Activation)      (None, 7, 7, 2048)   0           add_14[0][0]                     
__________________________________________________________________________________________________
res5b_branch2a (Conv2D)         (None, 7, 7, 512)    1049088     activation_43[0][0]              
__________

In [18]:
img=Image.open('./Rahul/IMG20180714170243.jpg',)

In [25]:
print(img.format,
img.size,
img.mode,
img.show)
img=img.resize((img_rows,img_cols))

None (224, 224) RGB <bound method Image.show of <PIL.Image.Image image mode=RGB size=224x224 at 0x12FDB05C948>>


In [30]:
img.size

(224, 224)

In [33]:
from numpy import asarray
data=asarray(img)
print(data.shape)

(224, 224, 3)


In [35]:
from numpy import expand_dims
dt=expand_dims(data,axis=0)

In [36]:
dt.shape

(1, 224, 224, 3)

In [37]:
modelnew.predict(dt)

array([[0.]], dtype=float32)